In [1]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import numpy as np
import pandas as pd
import random

In [8]:
df = pd.read_csv('/Users/gengyunxin/Documents/项目/traffic_model/test/data/normalized_data_72.csv', header=0)
Data = df.values
TS_num = Data.shape[0]
Data

array([[0.39798489, 0.43576826, 0.4256927 , ..., 0.49874055, 0.52141058,
        1.        ],
       [0.50125945, 0.61712846, 0.53400504, ..., 0.57178841, 0.58942065,
        1.        ],
       [0.58690176, 0.59949622, 0.697733  , ..., 0.38035264, 0.37783375,
        1.        ],
       ...,
       [0.12195122, 0.1358885 , 0.14285714, ..., 0.79094077, 0.79094077,
        0.        ],
       [0.77351916, 0.77351916, 0.79442509, ..., 0.81533101, 0.79442509,
        0.        ],
       [0.77351916, 0.76655052, 0.76655052, ..., 0.17421603, 0.16376307,
        0.        ]])

In [17]:
index = 0
# ts_length = 72
seq_len = 72
word_len = 6
ts_data = Data[:, 0:-1]
ts_processed = np.expand_dims(np.array(ts_data[index]), -1)
class_label = np.array([Data[index][-1]], dtype=int)

In [23]:
ts_length = ts_processed.shape[0]
num_words = int(ts_length / word_len)
num_words

12

In [24]:
bert_mask = np.zeros((seq_len,), dtype=int)
bert_mask[:ts_length] = 1
bert_mask

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [31]:
ts_processed

array([[0.39798489],
       [0.43576826],
       [0.4256927 ],
       [0.39042821],
       [0.40302267],
       [0.39798489],
       [0.38287154],
       [0.46095718],
       [0.45843829],
       [0.47607053],
       [0.50629723],
       [0.38035264],
       [0.5768262 ],
       [0.58690176],
       [0.44080605],
       [0.54911839],
       [0.45843829],
       [0.54156171],
       [0.52392947],
       [0.45088161],
       [0.50881612],
       [0.53652393],
       [0.47607053],
       [0.55163728],
       [0.4836272 ],
       [0.4836272 ],
       [0.48110831],
       [0.4836272 ],
       [0.44332494],
       [0.51889169],
       [0.41813602],
       [0.53148615],
       [0.51889169],
       [0.47858942],
       [0.4256927 ],
       [0.42821159],
       [0.45088161],
       [0.37783375],
       [0.50881612],
       [0.50881612],
       [0.44584383],
       [0.41309824],
       [0.38287154],
       [0.44332494],
       [0.42821159],
       [0.40806045],
       [0.40806045],
       [0.403

In [35]:
dimension = 1
ts_masking = ts_processed.copy()
mask = np.zeros((seq_len,), dtype=int)

for i in range(num_words):
    prob = random.random()
    if prob < 0.2:
        prob /= 0.2
        mask[(i * word_len):(i * word_len + 6)] = 1

        if prob < 0.5:
            ts_masking[(i * word_len):(i * word_len + 6), :] \
                += np.random.uniform(low=-0.5, high=0, size=(dimension,))
        else:
            ts_masking[(i * word_len):(i * word_len + 6), :] \
                += np.random.uniform(low=0, high=0.5, size=(dimension,))

mask

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1])

In [30]:
ts_masking


array([[0.39798489],
       [0.43576826],
       [0.4256927 ],
       [0.39042821],
       [0.40302267],
       [0.39798489],
       [0.38287154],
       [0.46095718],
       [0.45843829],
       [0.47607053],
       [0.50629723],
       [0.38035264],
       [0.3533097 ],
       [0.36338526],
       [0.21728955],
       [0.32560189],
       [0.23492179],
       [0.31804521],
       [0.52392947],
       [0.45088161],
       [0.50881612],
       [0.53652393],
       [0.47607053],
       [0.55163728],
       [0.4836272 ],
       [0.4836272 ],
       [0.48110831],
       [0.4836272 ],
       [0.44332494],
       [0.51889169],
       [0.41813602],
       [0.53148615],
       [0.51889169],
       [0.47858942],
       [0.4256927 ],
       [0.42821159],
       [0.45088161],
       [0.37783375],
       [0.50881612],
       [0.50881612],
       [0.44584383],
       [0.41309824],
       [0.38287154],
       [0.44332494],
       [0.42821159],
       [0.40806045],
       [0.40806045],
       [0.403

In [55]:
# file_path = '/Users/gengyunxin/Documents/项目/traffic_model/test/data/normalized_data_72.csv'
class PretrainDataset(Dataset):
    def __init__(self,
                 file_path='/Users/gengyunxin/Documents/项目/traffic_model/test/data/normalized_data_72.csv',
                 num_features = 1, seq_len=72, word_len=6):
        self.seq_len = seq_len
        self.word_len = word_len
        self.dimension = num_features

        df = pd.read_csv(file_path)
        self.Data = df.values # (5855,73)
        print("Loading data successfully!")
        self.TS_num = self.Data.shape[0]

    def __len__(self):
        return self.TS_num

    def __getitem__(self, index):
        ts_data = self.Data[:, 0:-1] # 不含label的数组 (5855,72)
        # Normalize
        # max = ts_data.max() # 400.0
        # min = ts_data.min() # 3.0
        # ts_data_normalized = (ts_data - min) / (max - min)

        ts_processed = np.expand_dims(np.array(ts_data[index]), -1) # 归一化后的数据 (72,1)
        class_label = np.array([self.Data[index][-1]], dtype=int) # label (1,)

        ts_length = ts_processed.shape[0] # 72
        num_words = int(ts_length / self.word_len)

        bert_mask = np.zeros((self.seq_len,), dtype=int)
        bert_mask[:ts_length] = 1 # 其实seq_len和ts_length长度相同，都是72,全1数组。这么写是为了应对长度不等的情况 (72,)

        # 随机噪声
        ts_masking, mask = self.random_masking(ts_processed, ts_length)

        output = {"bert_input": ts_masking, # 加噪声的时间序列 (72,1)
                  "bert_target": ts_processed, # 原来未加噪声的时间序列 (72,1)
                  "bert_mask": bert_mask, # 有数据的地方是1（长度ts_length）,其他地方是0（全长seq_len） (72,)
                  "loss_mask": mask, # 只计算加噪声位置的loss (72,),加噪声的位置是1,其余位置是0
                  "class_label": class_label # (1,)
                  }

        return {key: torch.from_numpy(value) for key, value in output.items()}


    def random_masking(self, ts, num_words):

        ts_masking = ts.copy()
        mask = np.zeros((self.seq_len,), dtype=int)

        for i in range(num_words):
            prob = random.random()
            if prob < 0.2:
                prob /= 0.2
                mask[(i * self.word_len):(i * self.word_len + self.word_len)] = 1

                if prob < 0.5:
                    ts_masking[(i * self.word_len):(i * self.word_len + self.word_len), :] \
                        += np.random.uniform(low=-0.5, high=0, size=(self.dimension,))
                else:
                    ts_masking[(i * self.word_len):(i * self.word_len + self.word_len), :] \
                        += np.random.uniform(low=0, high=0.5, size=(self.dimension,))

        return ts_masking, mask

In [54]:
np.random.seed(0)

In [ ]:
class DatasetWrapper(object):
    """划分训练集和验证集"""
    def __init__(self, batch_size, valid_ratio, data_path, num_features, max_length):
        self.batch_size = batch_size
        self.valid_ratio = valid_ratio
        self.data_path = data_path
        self.num_features = num_features
        self.max_length = max_length
        
    def get_data_loaders(self):
        dataset = PretrainDataset(self.data_path, self.num_features, self.max_length)
        train_loader, valid_loader = self.get_train_valid_data_loaders(dataset)
        return train_loader, valid_loader
    
    def get_train_valid_data_loaders(self, dataset):
        num_data = len(dataset) # 1463
        indices = list(range(num_data))
        np.random.shuffle(indices)
        
        valid_split = int(np.floor(self.valid_ratio * num_data))
        print('training samples: %d, validation samples: %d' % (num_data-valid_split, valid_split))
        train_idx, valid_idx = indices[valid_split:], indices[:valid_split]
        
        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)
        
        train_loader = DataLoader(dataset, batch_size=self.batch_size, sampler=train_sampler,
                                 drop_last=True)
        valid_loader = DataLoader(dataset, batch_size=self.batch_size, sampler=valid_idx_sampler,
                                 drop_last=True)
        
        return train_loader, valid_loader